# convert_raw

This notebook applies various processing routines and conversions to the raw FlowMow2 data to generate L1B data.

#### Convert Paros values to temperature and pressure

Paros pressure values are in psia, which according to the [Paros manual](../docs/G8203_Digiquartz_Broadband_Pressure_Transducers_and_Depth_Sensors_with_Frequency_Outputs.pdf) can be converted to pascals by multipyling by 9806.650, or to m of H2O by multiplying by 0.7030696. Temperature is in degrees C. The SBE3 is likely the better instrument to use for temperature. The instument [calibration constants](../data/info/paros_cals.yaml) are extracted directly from the raw DAT files.

In [ ]:
import pandas as pd
import yaml
import flowmow

In [ ]:
paros = pd.read_hdf('../data/interim/paros_raw.h5', 'table')

In [ ]:
with open('../data/info/paros_cals.yaml') as f:
    paros_cals = yaml.load(f)

In [ ]:
paros['temp'], paros['pressure'] = flowmow.convert_paros(paros.eta, paros.tau, **paros_cals)
paros.head()

In [ ]:
%matplotlib inline
fig = paros.temp.plot(title='temp');

In [ ]:
((paros.pressure)*0.7030696).plot(title='Approximage Depth');

In [ ]:
# save to hdf5
paros.to_hdf('../data/interim/paros_proc.h5', 'table', append=False, data_columns=True)
del paros

#### Convert SBE3 values to temperature

The Seabird SBE3 [calibration constants](../data/info/sbe3_cals.yaml) are from the [official calibration sheets](../docs/SBE03_2014_cals.pdf) which also contain the conversion equations. More info on the SBE3 [here](../docs/datasheet-03plus-May15.pdf).

In [ ]:
sbe3 = pd.read_hdf('../data/interim/sbe3_raw.h5', 'table')

In [ ]:
with open('../data/info/sbe3_cals.yaml') as f:
    sbe3_cals = yaml.load(f)

In [ ]:
# sbe3 2265 was on the stinger and was recorded as counts_0
sbe3['temp_stinger'] = flowmow.convert_sbe3(sbe3.counts_0, **sbe3_cals[2265])
sbe3['temp_top'] = flowmow.convert_sbe3(sbe3.counts_1, **sbe3_cals[2446])
sbe3.head()

In [ ]:
sbe3[['temp_stinger','temp_top']].plot();

In [ ]:
# save to hdf5
sbe3.to_hdf('../data/interim/sbe3_proc.h5', 'table', append=False, data_columns=True)
del sbe3

#### Get UTM northing and easting values

In [ ]:
from pyproj import Proj

In [ ]:
nav = pd.read_hdf('../data/interim/nav_raw.h5', 'table')

In [ ]:
p = Proj(proj='utm', zone=9)

In [ ]:
nav['x'], nav['y'] = p(nav.lon.values, nav.lat.values)

In [ ]:
nav.head()

#### Build a rotated coordinate system

In [ ]:
import numpy as np

In [ ]:
omega = 29 # rotate by omega degrees
theta = np.deg2rad(omega)
v = np.transpose(np.array([nav.x.values, nav.y.values]))
R = np.array([[np.cos(theta),-np.sin(theta)], [np.sin(theta),np.cos(theta)]])
vr = v.dot(R)

# add to scc and store
nav['xrot'] = vr[:,0]
nav['yrot'] = vr[:,1]
nav.head()

In [ ]:
# save to hdf5
nav.to_hdf('../data/interim/nav_proc.h5', 'table', append=False, data_columns=True)

#### Show rotated survey lines

In [ ]:
# subsample the vehicle locations to make a smaller plot
x = nav.iloc[::10].x
y = nav.iloc[::10].y
xrot = nav.iloc[::10].xrot
yrot = nav.iloc[::10].yrot

In [ ]:
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

# create a new figure
p = figure(tools='pan,wheel_zoom,box_zoom,reset', match_aspect=True, output_backend="webgl", lod_threshold=100)

# add a circle renderer with vecorized colors and sizes
shiftx = -15
shifty = -680

p.circle(x-x.min()+shiftx, y-y.min()+shifty, radius=1, fill_color='blue', fill_alpha=0.6, line_color=None)
p.circle(xrot-xrot.min()+shiftx, yrot-yrot.min()+shifty, radius=1, fill_color='green',
         fill_alpha=0.6, line_color=None)

# show the results
show(p)